In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential

In [ ]:
epochs = 3
img_height = 200
img_width = 200
data_dir = "data/asl_alphabet_train"
test_dir = "data/asl_alphabet_test"

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
classes = train_ds.class_names

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
x = os.listdir(test_dir)
class_enum = enumerate(train_ds.class_names)
class_dict = dict((j, i) for i, j in class_enum)
image_array = []
label_array = []
for item in x:
  if item.lower().endswith('.jpg'):
    image = tf.io.read_file(test_dir + '/' + item)
    image = tf.image.decode_image(image, channels = 3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=(200, 200))
    image_array.append(image)
    label = class_dict.get(item[0])
    label_array.append(label)

In [ ]:
image_array = np.asarray(image_array)
label_array = np.asarray(label_array)

In [ ]:
def plot_his(history):
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epochs')
  plt.legend(['train', 'validation'])
  plt.show()

  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epochs')
  plt.legend(['train', 'validation'])
  plt.show()

In [ ]:
model = Sequential()

model.add(tf.keras.layers.Conv2D(32, (5, 5), 
                 strides = 3,
                 padding = 'Same',
                 activation = 'relu',
                 input_shape = (200, 200, 3)))

model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, (3, 3),
                 strides = 2,
                 padding = 'Same',
                 activation = 'relu'))

model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(108, activation = 'relu'))

model.add(tf.keras.layers.Dense(108, activation = 'relu'))

model.add(tf.keras.layers.Dense(26))

model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate = 0.1),
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics = ['accuracy'])

In [ ]:
filepath = "data/best_model.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,
              monitor='val_accuracy', verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

history_3 = model.fit(train_ds, validation_data=val_ds, epochs=epochs,
                    callbacks = callbacks_list)

In [ ]:
plot_his(history_3)

In [ ]:
model.evaluate(image_array, label_array)